# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
main_df = pd.read_csv("output_data/main_df.csv", index_col=0)
main_df.head()

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Nikolskoye,RU,59.70,30.79,8.33,93,53,1.00,09/30/2020
1,Mezhdurechensk,RU,53.69,88.06,1.46,92,100,2.87,09/30/2020
2,Kulhudhuffushi,MV,6.62,73.07,27.58,73,87,4.51,09/30/2020
3,Lobatse,BW,-25.22,25.67,16.00,82,92,10.30,09/30/2020
4,Ushuaia,AR,-54.80,-68.30,-1.00,80,90,12.30,09/30/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)
locations = main_df[["Lat", "Long"]]
weights = main_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False))
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
    
hotel_df = main_df.loc[(main_df["Max Temp"] > 25) & (main_df["Max Temp"] < 30) & (main_df["Wind Speed"] < 15) & (main_df["Cloudiness"] < 60)]             
hotel_df

,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Date
29,Chimoré,BO,-16.98,-65.13,27.26,53,29,0.70,09/30/2020
31,George Town,MY,5.41,100.34,28.00,88,20,1.50,09/30/2020
33,Atuona,PF,-9.80,-139.03,25.88,74,1,9.00,09/30/2020
38,Bengkulu,ID,-3.80,102.27,26.97,78,19,2.62,09/30/2020
49,Butaritari,KI,3.07,172.79,27.75,74,23,8.44,09/30/2020
51,Mackay,AU,-21.15,149.20,26.67,57,0,3.60,09/30/2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
Hotel_name = []
exception = 1
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    params["location"] = f"{lat},{long}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel = response["results"][0]["name"]
        Hotel_name.append(hotel)
    except:
        print(f"This is the #{exception} exception")
        Hotel_name.append("none")
        exception = exception + 1
        pass


This is the #1 exception


In [28]:
#hotel_df.insert(8, "Hotel Name", Hotel_name)
hotel_data = hotel_df.loc[hotel_df["Hotel Name"]!="none"]
new_hotel_data = hotel_data.drop(["Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"], axis=1)
new_hotel_data

,City,Country,Lat,Long,Hotel Name
29,Chimoré,BO,-16.98,-65.13,Hotel Copacabana
31,George Town,MY,5.41,100.34,The Prestige Hotel Penang
33,Atuona,PF,-9.80,-139.03,CHALETS HANAKEE
38,Bengkulu,ID,-3.80,102.27,Mercure Bengkulu
51,Mackay,AU,-21.15,149.20,Mantra Mackay


In [29]:
a = new_hotel_data["Lat"].tolist()
b = new_hotel_data["Long"].tolist()
x = zip(a, b)
coordinates = list(x)
coordinates

[(-16.98, -65.13),
 (5.41, 100.34),
 (-9.8, -139.03),
 (-3.8, 102.27),
 (-21.15, 149.2)]

In [30]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_data.iterrows()]
#locations = hotel_df[["Lat", "Long"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates,info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))